In [1]:
!ffmpeg -version

'ffmpeg' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
from pathlib import Path
import os
import csv
import time

import numpy as np
import torch
from ultralytics import YOLO

# Root paths (work relative to repo root)
REPO_ROOT = Path.cwd().resolve()
DATA_ROOT = REPO_ROOT / "data" / "soccer_side"
RESULTS_ROOT = REPO_ROOT / "results" / "yolov11_botsort"

RESULTS_ROOT.mkdir(parents=True, exist_ok=True)

print(f"Repo root: {REPO_ROOT}")
print(f"Data root: {DATA_ROOT}")
print(f"Results root: {RESULTS_ROOT}")

# Device selection
if torch.cuda.is_available():
    DEVICE = "cuda"
else:
    DEVICE = "cpu"

print("Using device:", DEVICE)


Repo root: C:\Users\rishi\OneDrive\College\CS543\sports-data-tracker
Data root: C:\Users\rishi\OneDrive\College\CS543\sports-data-tracker\data\soccer_side
Results root: C:\Users\rishi\OneDrive\College\CS543\sports-data-tracker\results\yolov11_botsort
Using device: cpu


In [2]:
# Pre-process videos: convert to fixed format using ffmpeg
# This ensures compatibility with video processing libraries
import subprocess
import glob
# Use specific ffmpeg path
ffmpeg_path = r"C:\Users\rishi\ffmpeg\bin\ffmpeg.exe"

# Check if ffmpeg exists at the specified path
if not os.path.exists(ffmpeg_path):
    print(f"[ERROR] ffmpeg not found at: {ffmpeg_path}")
    print("Please update the ffmpeg_path variable in this cell with the correct path.")
    print("Skipping video preprocessing.")
else:
    print(f"Using ffmpeg at: {ffmpeg_path}")
    print("Pre-processing videos to fixed format...")
    videos_processed = 0
    videos_skipped = 0
    videos_failed = 0

    for vid in glob.glob("data/soccer_side/**/*.mp4", recursive=True):
        # Skip if it's already a _fixed.mp4 file
        if "_fixed.mp4" in vid:
            continue
        
        fixed = vid.replace(".mp4", "_fixed.mp4")
        
        if not os.path.exists(fixed):
            print(f"Processing: {vid} -> {fixed}")
            try:
                result = subprocess.run([
                    ffmpeg_path, "-y", "-i", vid,
                    "-c:v", "libx264", "-pix_fmt", "yuv420p",
                    "-c:a", "aac", fixed
                ], capture_output=True, text=True, timeout=300)  # 5 minute timeout per video
                
                if result.returncode == 0:
                    videos_processed += 1
                    print(f"  ✓ Success")
                else:
                    videos_failed += 1
                    print(f"  ✗ Failed: {result.stderr[:200]}")  # Show first 200 chars of error
            except subprocess.TimeoutExpired:
                videos_failed += 1
                print(f"  ✗ Timeout (video too long or stuck)")
            except Exception as e:
                videos_failed += 1
                print(f"  ✗ Error: {str(e)}")
        else:
            videos_skipped += 1

    print(f"\nPre-processing complete:")
    print(f"  - Processed: {videos_processed} video(s)")
    print(f"  - Already fixed: {videos_skipped} video(s)")
    if videos_failed > 0:
        print(f"  - Failed: {videos_failed} video(s)")


Using ffmpeg at: C:\Users\rishi\ffmpeg\bin\ffmpeg.exe
Pre-processing videos to fixed format...
Processing: data/soccer_side\test\F_20220220_1_1680_1710\img1.mp4 -> data/soccer_side\test\F_20220220_1_1680_1710\img1_fixed.mp4
  ✓ Success
Processing: data/soccer_side\test\F_20220220_1_1710_1740\img1.mp4 -> data/soccer_side\test\F_20220220_1_1710_1740\img1_fixed.mp4
  ✓ Success
Processing: data/soccer_side\test\F_20220220_1_1740_1770\img1.mp4 -> data/soccer_side\test\F_20220220_1_1740_1770\img1_fixed.mp4
  ✓ Success
Processing: data/soccer_side\test\F_20220220_1_1770_1800\img1.mp4 -> data/soccer_side\test\F_20220220_1_1770_1800\img1_fixed.mp4
  ✓ Success
Processing: data/soccer_side\test\F_20220220_1_1800_1830\img1.mp4 -> data/soccer_side\test\F_20220220_1_1800_1830\img1_fixed.mp4
  ✓ Success
Processing: data/soccer_side\test\F_20220220_1_1830_1860\img1.mp4 -> data/soccer_side\test\F_20220220_1_1830_1860\img1_fixed.mp4
  ✓ Success
Processing: data/soccer_side\test\F_20220220_1_1860_1890\im

In [3]:
VIDEO_EXTS = {".mp4", ".mkv", ".avi", ".mov"}

def discover_sequences(data_root: Path):
    """
    Discover all sequences under data_root that:
    - Have a gt/gt.txt file
    - Have at least one video file in the same parent directory as gt/
    
    Prefers _fixed.mp4 files over regular .mp4 files for better compatibility.
    """
    seqs = []

    for gt_file in data_root.rglob("gt.txt"):
        gt_dir = gt_file.parent              # .../seq_name/gt
        seq_dir = gt_dir.parent              # .../seq_name
        video_files = [
            p for p in seq_dir.iterdir()
            if p.is_file() and p.suffix.lower() in VIDEO_EXTS
        ]

        if not video_files:
            # No video attached to this gt; skip
            continue

        # Prefer _fixed.mp4 files over regular .mp4 files
        fixed_videos = [v for v in video_files if "_fixed.mp4" in v.name]
        if fixed_videos:
            # Use _fixed.mp4 if available
            video_files = fixed_videos
        else:
            # Otherwise prefer .mp4 over other formats
            mp4_videos = [v for v in video_files if v.suffix.lower() == ".mp4"]
            if mp4_videos:
                video_files = mp4_videos

        # If multiple videos exist, pick the first alphabetically and warn
        video_files.sort()
        video_path = video_files[0]
        if len(video_files) > 1:
            print(f"[WARN] Multiple videos in {seq_dir}, using {video_path.name}")

        seqs.append(
            {
                "name": seq_dir.name,
                "seq_dir": seq_dir,
                "gt_path": gt_file,
                "video_path": video_path,
            }
        )

    return seqs

sequences = discover_sequences(DATA_ROOT)

print(f"Discovered {len(sequences)} sequence(s) with video + gt:")
for s in sequences:
    print(f"  - {s['name']}: video={s['video_path'].relative_to(REPO_ROOT)}, gt={s['gt_path'].relative_to(REPO_ROOT)}")


Discovered 22 sequence(s) with video + gt:
  - F_20220220_1_1680_1710: video=data\soccer_side\test\F_20220220_1_1680_1710\img1_fixed.mp4, gt=data\soccer_side\test\F_20220220_1_1680_1710\gt\gt.txt
  - F_20220220_1_1710_1740: video=data\soccer_side\test\F_20220220_1_1710_1740\img1_fixed.mp4, gt=data\soccer_side\test\F_20220220_1_1710_1740\gt\gt.txt
  - F_20220220_1_1740_1770: video=data\soccer_side\test\F_20220220_1_1740_1770\img1_fixed.mp4, gt=data\soccer_side\test\F_20220220_1_1740_1770\gt\gt.txt
  - F_20220220_1_1770_1800: video=data\soccer_side\test\F_20220220_1_1770_1800\img1_fixed.mp4, gt=data\soccer_side\test\F_20220220_1_1770_1800\gt\gt.txt
  - F_20220220_1_1800_1830: video=data\soccer_side\test\F_20220220_1_1800_1830\img1_fixed.mp4, gt=data\soccer_side\test\F_20220220_1_1800_1830\gt\gt.txt
  - F_20220220_1_1830_1860: video=data\soccer_side\test\F_20220220_1_1830_1860\img1_fixed.mp4, gt=data\soccer_side\test\F_20220220_1_1830_1860\gt\gt.txt
  - F_20220220_1_1860_1890: video=data\

In [4]:
# You can switch to 'yolo11s.pt' or larger if you want better accuracy and have GPU resources
YOLO_WEIGHTS = "yolo11n.pt"   # or "yolo11s.pt"

print(f"Loading YOLOv11 model from weights: {YOLO_WEIGHTS}")
yolo_model = YOLO(YOLO_WEIGHTS)
yolo_model.to(DEVICE)

# COCO class index for 'person' is 0
PERSON_CLASS_ID = 0


Loading YOLOv11 model from weights: yolo11n.pt


In [5]:
def track_sequence_with_yolo_botsort(
    model: YOLO,
    video_path: Path,
    seq_name: str,
    save_dir: Path,
    classes=(PERSON_CLASS_ID,),
    conf: float = 0.15,
    iou: float = 0.5,
    device: str = DEVICE,
    save_video: bool = True,
):
    """
    Run YOLOv11 + BoT-SORT on a single video and export detections in MOT format.
    Optionally saves annotated video with tracking visualizations in MP4 format.

    Parameters
    ----------
    model : YOLO
        Ultralytics YOLO model instance.
    video_path : Path
        Path to the input video.
    seq_name : str
        Name of the sequence (used for result filename).
    save_dir : Path
        Directory to write MOT-format result file into.
    classes : tuple
        Tuple of class IDs to keep (default: person only).
    conf : float
        Confidence threshold.
    iou : float
        IoU threshold for NMS.
    device : str
        "cuda" or "cpu".
    save_video : bool
        If True, save annotated video with tracking visualizations.
    """
    import cv2
    
    save_dir.mkdir(parents=True, exist_ok=True)
    mot_out_path = save_dir / f"{seq_name}.txt"
    video_out_dir = save_dir / "videos"
    video_out_dir.mkdir(exist_ok=True)
    video_out_path = video_out_dir / f"{seq_name}_tracked.mp4"

    # Open CSV writer for MOTChallenge-style output
    f = open(mot_out_path, "w", newline="")
    writer = csv.writer(f)

    frame_idx = 0
    t_start = time.time()

    # Initialize video writer if saving video
    video_writer = None
    temp_video_path = None
    if save_video:
        # Get video properties from input video
        cap = cv2.VideoCapture(str(video_path))
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        cap.release()
        
        # Write to temporary file first, then re-encode with ffmpeg for proper H.264
        temp_video_path = video_out_path.parent / f"{video_out_path.stem}_temp.avi"
        # Use XVID for temporary file (works reliably on Windows)
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        video_writer = cv2.VideoWriter(str(temp_video_path), fourcc, fps, (width, height))

    # Run tracking - use streaming mode to process frame by frame
    # This allows us to manually write MP4 videos
    results_generator = model.track(
        source=str(video_path),
        tracker="botsort.yaml",
        conf=conf,
        iou=iou,
        classes=list(classes),
        device=device,
        stream=True,  # Use streaming to get annotated frames
        verbose=False,
        save=False,  # Don't use Ultralytics' built-in saving (saves as AVI)
    )

    # Process results to write MOT format and save video frames
    num_det_total = 0
    for res in results_generator:
        frame_idx += 1  # MOT frames are 1-based
        boxes = res.boxes

        # Get annotated frame (with bounding boxes drawn)
        annotated_frame = res.plot()  # This returns the frame with annotations
        
        # Write frame to video if saving
        if save_video and video_writer is not None:
            # Convert BGR to RGB (plot() returns RGB, OpenCV needs BGR)
            frame_bgr = cv2.cvtColor(annotated_frame, cv2.COLOR_RGB2BGR)
            video_writer.write(frame_bgr)

        if boxes is None or boxes.id is None:
            continue

        ids = boxes.id.cpu().numpy().astype(int)
        xyxy = boxes.xyxy.cpu().numpy()
        confs = boxes.conf.cpu().numpy()

        for trk_id, (x1, y1, x2, y2), c in zip(ids, xyxy, confs):
            w = x2 - x1
            h = y2 - y1

            # MOT format: frame, id, bb_left, bb_top, bb_width, bb_height, conf, -1, -1, -1
            writer.writerow([
                frame_idx,
                int(trk_id),
                float(x1),
                float(y1),
                float(w),
                float(h),
                float(c),
                -1,
                -1,
                -1,
            ])
            num_det_total += 1

    f.close()
    if save_video and video_writer is not None:
        video_writer.release()
        
        # Re-encode to H.264 MP4 using ffmpeg for better compatibility
        if temp_video_path and temp_video_path.exists():
            import subprocess
            import shutil
            
            # Try to find ffmpeg
            ffmpeg_cmd = None
            for possible_path in [
                "ffmpeg",
                r"C:\Users\rishi\ffmpeg\bin\ffmpeg.exe",
                shutil.which("ffmpeg")
            ]:
                if possible_path and (possible_path == "ffmpeg" or (isinstance(possible_path, str) and Path(possible_path).exists())):
                    ffmpeg_cmd = possible_path
                    break
            
            if ffmpeg_cmd:
                try:
                    # Use ffmpeg to re-encode to H.264 MP4
                    subprocess.run([
                        ffmpeg_cmd, "-y", "-i", str(temp_video_path),
                        "-c:v", "libx264", "-preset", "medium", "-crf", "23",
                        "-c:a", "copy",  # Copy audio if present
                        "-pix_fmt", "yuv420p",  # Ensure compatibility
                        str(video_out_path)
                    ], check=True, capture_output=True)
                    # Remove temporary file
                    temp_video_path.unlink()
                except (subprocess.CalledProcessError, FileNotFoundError) as e:
                    # If ffmpeg fails, just rename the temp file
                    print(f"[{seq_name}] Warning: ffmpeg re-encoding failed, using temporary file: {e}")
                    if video_out_path.exists():
                        video_out_path.unlink()
                    temp_video_path.rename(video_out_path)
            else:
                # No ffmpeg found, just rename temp file
                if video_out_path.exists():
                    video_out_path.unlink()
                temp_video_path.rename(video_out_path)
    
    t_end = time.time()
    elapsed = t_end - t_start

    print(f"[{seq_name}] Wrote {num_det_total} detections to {mot_out_path}")
    if save_video and video_out_path.exists():
        print(f"[{seq_name}] Saved annotated video to {video_out_path}")
    print(f"[{seq_name}] Time: {elapsed:.2f}s, FPS ~ {frame_idx / elapsed if elapsed > 0 else 0:.2f}")

    return mot_out_path


In [ ]:
mot_results_dir = RESULTS_ROOT / "mot_outputs"
mot_results_dir.mkdir(parents=True, exist_ok=True)

result_paths = {}

for seq in sequences:
    name = seq["name"]
    video_path = seq["video_path"]

    print(f"\n=== Running YOLOv11 + BoT-SORT on sequence: {name} ===")
    print(f"Video: {video_path}")

    out_path = track_sequence_with_yolo_botsort(
        model=yolo_model,
        video_path=video_path,
        seq_name=name,
        save_dir=mot_results_dir,
        classes=(PERSON_CLASS_ID,),
        conf=0.15,
        iou=0.5,
        device=DEVICE,
    )
    result_paths[name] = out_path

print("\nAll sequences processed. Output MOT files:")
for name, path in result_paths.items():
    print(f"  - {name}: {path.relative_to(REPO_ROOT)}")



=== Running YOLOv11 + BoT-SORT on sequence: F_20220220_1_1680_1710 ===
Video: C:\Users\rishi\OneDrive\College\CS543\sports-data-tracker\data\soccer_side\test\F_20220220_1_1680_1710\img1_fixed.mp4
[F_20220220_1_1680_1710] Wrote 69 detections to C:\Users\rishi\OneDrive\College\CS543\sports-data-tracker\results\yolov11_botsort\mot_outputs\F_20220220_1_1680_1710.txt
[F_20220220_1_1680_1710] Saved annotated video to C:\Users\rishi\OneDrive\College\CS543\sports-data-tracker\results\yolov11_botsort\mot_outputs\videos\F_20220220_1_1680_1710_tracked.mp4
[F_20220220_1_1680_1710] Time: 115.60s, FPS ~ 6.49

=== Running YOLOv11 + BoT-SORT on sequence: F_20220220_1_1710_1740 ===
Video: C:\Users\rishi\OneDrive\College\CS543\sports-data-tracker\data\soccer_side\test\F_20220220_1_1710_1740\img1_fixed.mp4
[F_20220220_1_1710_1740] Wrote 0 detections to C:\Users\rishi\OneDrive\College\CS543\sports-data-tracker\results\yolov11_botsort\mot_outputs\F_20220220_1_1710_1740.txt
[F_20220220_1_1710_1740] Saved a